<a href="https://colab.research.google.com/github/pportella23/Sentiment-Analysis/blob/master/Social_Media_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Atividade Prática de Análise de Sentimento**

## **PARTE I**

### **Descrição do dataset**

Dado um dataset de treino composto por tweets e rótulos, onde rótulo ‘1’ denota que o tweet tem conteúdo negativo e rótulo ‘0’ denota que o tweet tem conteúdo positivo, nosso objetivo é tentar prever os rótulos para um dataset de teste. 

O dataset no seu formato puro (raw) é composto de:
*   id : O id associado com cada tweet no dataset.
*   tweets : Os tweets coletados de várias fontes e com sentimentos negativos ou positivos associados à eles.
*   Rótulo : Valor binário de representação de sentimento. Sendo ‘0’ se o tweet possui um conteúdo positivo e ‘1’ se o tweet possui um conteúdo negativo.




O primeiro passo é importar todos os pacotes necessários para trabalhar com o pré-processamento desses dados coletados no dataset

In [0]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

Agora vamos efetuar a leitura desses datasets, que estão contidos em um repositório do github como arquivos csv, e salvá-los em cópias dos datasets originais por segurança

In [0]:
# Leitura do dataset de treino
train = pd.read_csv('https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/train.csv')

# Leitura do dataset de teste
test = pd.read_csv('https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/test.csv')

# Cópia do dataset de treino original
train_original=train.copy()

# Cópia do dataset de teste original
test_original=test.copy()

É possível obter um overview dos datasets e suas estruturas (conforme explicado no enunciado)

In [0]:
# Comandos para obter um overview do dataset e averiguar sua estrutura

# dataset de treino (3 colunas)
train

# dataset de teste (2 colunas)
test

OBS:

Note que se você tentar visualizar o dataset de teste (test) só terá duas colunas: id e tweet.

Isso se dá, porque ainda não fizemos a predição dos rótulos.

Agora, se a visualização for do dataset de treino (train) terá as três colunas comentadas no enunciado: id, tweet e rótulo.



### **Pré-Processamento dos dados**

Vamos começar combinando os dois datasets por meio de uma anexação de todo o dataset de teste no final do dataset de treino 

In [0]:
combine = train.append(test,ignore_index=True,sort=True)

# Podemos visualizar, para garantir que está correto, o topo e o final do dataset que foi gerado da combinação

# O topo (primeiras 5 linhas) do dataset são correspondentes as informações que eram do dataset de treino
combine.head()

# O final (últimas 5 linhas) do dataset são correspondentes as informações que eram do dataset de teste
combine.tail()

OBS:

Perceba que a coluna `label` das linhas finais estão populadas com o valor `NaN`, justamente porque essas linhas são referentes aos dados do dataset de teste que não possui rótulos ainda

Nosso próximo passo será remover os nomes de usuário dos tweets, visto que não contribuem em nada para a análise 

In [0]:
# Para isso criamos uma função que recebe o texto e o que será removido dele

def remove_pattern(text,pattern):
    
    # re.findall() acha o conteúdo a ser removido e coloca em uma lista r

    r = re.findall(pattern,text)
    
    # re.sub() remove o conteúdo das sentenças do
    
    for i in r:
        text = re.sub(i,"",text)
    
    return text

In [0]:
# E aqui então utilizamos a função recém criada passando os dados que desejamos

# Usamos np.vectorize porque é muito mais otimizado quando trata-se de datasets grandes

combine['Tidy_Tweets'] = np.vectorize(remove_pattern)(combine['tweet'], "@[\w]*")

# E podemos visualizar novamente o dataset, dessa vez com mais uma coluna adicionada

combine.head()

Nesse próximo passo iremos remover pontuações, números, caracteres especiais e stop words.

In [0]:
# Remoção das pontuações e caracteres especiais

combine['Tidy_Tweets'] = combine['Tidy_Tweets'].str.replace("[^a-zA-Z#]", " ")

# Remoção das stop words (palavras com menos de 3 letras)

combine['Tidy_Tweets'] = combine['Tidy_Tweets'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# Agora podemos visualizar o dataset com a coluna Tidy atualizada

combine.head(10)

Outra etapa importante é realizar a tokenização, que seria dividir a sentença em cada uma de suas palavras. Ou seja, cada palavra da sentença receberá uma posição diferente em um vetor criado.

In [0]:
# Comando de tokenização

tokenized_tweet = combine['Tidy_Tweets'].apply(lambda x: x.split())

# Visualização da lista de vetores com os tweets tokenizados

tokenized_tweet.head()

Essa tokenização foi realizada justamente para facilitar a etapa de stemming. Nessa etapa iremos remover derivações e formas flexionadas das palavras, pois na maioria das vezes temos muitas palavras com contextos relacionados e que contém significados muito semelhantes (por exemplo: nation, national, nationality). Portanto, desejamos extrair apenas o radical dessas palavras para facilitar o trabalho

In [0]:
# Faremos a importação do método de stemming da lib nltk
# nltk é uma lib de PLN muito conhecida do Python

from nltk import PorterStemmer

ps = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [ps.stem(i) for i in x])

tokenized_tweet.head()

Agora que já extraímos os radicais das palavras, vamos juntar esses tokens novamente em sentenças

In [0]:
# Um for que passa pelos tokens e vai juntando-os novamente no vetor

for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

combine['Tidy_Tweets'] = tokenized_tweet

combine.head()

E dessa forma, temos o pré-processamento do nosso dataset realizado! :)

### **Visualização dos dados**

Nessa seção falaremos de visualização de dados, um fator muito importante porque nos dá uma ideia aproximada sobre o dataset que tratamos antes de aplicar os modelos de aprendizado de máquina. Falaremos em especial de uma técnica bastante conhecida que é a `WordCloud`, ela nada mais é do que uma visualização onde as palavras que aparecem com maior frequência são representadas com um tamanho maior e as menos frequentes com tamanhos menores.



In [0]:
# O Python já possui um pacote bastante utilizado de wordcloud

from wordcloud import WordCloud,ImageColorGenerator
from PIL import Image
import urllib
import requests

# Basta armazenar as palavras

all_words_positive = ' '.join(text for text in combine['Tidy_Tweets'][combine['label']==0])

# E criar a "nuvem"

# Combinando a imagem de fundo com as palavras do dataset

Mask = np.array(Image.open(requests.get('http://clipart-library.com/image_gallery2/Twitter-PNG-Image.png', stream=True).raw))

# Aqui realizamos manipulação da imagem e das palavras com a lib do wordcloud importado

image_colors = ImageColorGenerator(Mask)

# E agora usamos a função WordCloud para gerar a imagem 

wc = WordCloud(background_color='black', height=1500, width=4000,mask=Mask).generate(all_words_positive)


E agora vamos exibir a imagem que geramos da "nuvem" de palavras positivas

In [0]:
# Tamanho da imagem gerada

plt.figure(figsize=(10,20))

# Aqui nós recolorimos as palavras do dataset com a cor que era da imagem

plt.imshow(wc.recolor(color_func=image_colors),interpolation="hamming")

plt.axis('off')
plt.show()

Vamos criar também a "nuvem" das palavras negativas

In [0]:
# Armazenando as palavras

all_words_negative = ' '.join(text for text in combine['Tidy_Tweets'][combine['label']==1])

# Combinando a imagem de fundo com as palavras do dataset

Mask = np.array(Image.open(requests.get('http://clipart-library.com/image_gallery2/Twitter-PNG-Image.png', stream=True).raw))

# Aqui realizamos manipulação da imagem e das palavras com a lib do wordcloud importado

image_colors = ImageColorGenerator(Mask)

# E agora usamos a função WordCloud para gerar a imagem 

wc = WordCloud(background_color='black', height=1500, width=4000,mask=Mask).generate(all_words_negative)

E ver como ela fica exibida

In [0]:
# Tamanho da imagem gerada

plt.figure(figsize=(10,20))

# Aqui nós recolorimos as palavras do dataset com a cor que era da imagem

plt.imshow(wc.recolor(color_func=image_colors),interpolation="gaussian")

plt.axis('off')
plt.show()

### **Outras manipulações de dados relevantes**

Levando em consideração que extraímos os dados do Twitter temos ainda o impacto causado pelas hashtags, portanto devemos extraí-las e analisá-las

In [0]:
# Função que encontra e retorna as hashtags dos tweets

def Hashtags_Extract(x):
    hashtags=[]
    
    # Loop over the words in the tweet
    for i in x:
        ht = re.findall(r'#(\w+)',i)
        hashtags.append(ht)
    
    return hashtags

Depois de extrair essas hashtags devemos colocá-las em alguma estrutura de dados (lista)

In [0]:
# Lista aninhada de hashtags dos tweets positivos

ht_positive = Hashtags_Extract(combine['Tidy_Tweets'][combine['label']==0])

ht_positive

Podemos desaninhá-las para facilitar uma manipulação posterior 

In [0]:
# Desaninhando a lista

ht_positive_unnest = sum(ht_positive,[])

E vamos realizar os mesmos passos para os tweets negativos

In [0]:
# Lista aninhada de hashtags dos tweets negativos

ht_negative = Hashtags_Extract(combine['Tidy_Tweets'][combine['label']==1])

ht_negative

Desaninhando a lista negativa também

In [0]:
# Desaninhando a lista

ht_negative_unnest = sum(ht_negative,[])

Por fim, podemos voltar a trabalhar com visualização de dados por meio de gráficos que exibam essas informações coletadas

### **Plotagem de gráficos**

Começaremos plotando um gráfico em barras dos tweets positivos no dataset

In [0]:
# Contamos a frequência das palavras positivas

word_freq_positive = nltk.FreqDist(ht_positive_unnest)

word_freq_positive

Criamos um dataframe para as palavras de uso mais frequente nas hashtags

In [0]:
# Criando o dataframe de palavras positivas das hashtags

df_positive = pd.DataFrame({'Hashtags':list(word_freq_positive.keys()),'Count':list(word_freq_positive.values())})

df_positive.head(10)

E por fim geramos o gráfico com essas informações

In [0]:
# Plotagem do gráfico das palavras positivas

df_positive_plot = df_positive.nlargest(20,columns='Count')

sns.barplot(data=df_positive_plot,y='Hashtags',x='Count')
sns.despine()

Agora um gráfico em barras dos tweets negativos no dataset

In [0]:
# Contamos a frequência das palavras negativas

word_freq_negative = nltk.FreqDist(ht_negative_unnest)

word_freq_negative

Criamos um dataframe para as palavras de uso mais frequente nas hashtags

In [0]:
# Criando o dataframe de palavras negativas das hashtags

df_negative = pd.DataFrame({'Hashtags':list(word_freq_negative.keys()),'Count':list(word_freq_negative.values())})

df_negative.head(10)

Por fim geramos o gráfico com essas informações

In [0]:
# Plotagem do gráfico das palavras negativas

df_negative_plot = df_negative.nlargest(20,columns='Count') 

sns.barplot(data=df_negative_plot,y='Hashtags',x='Count')
sns.despine()

## **PARTE II**

### **Bag-of-words**

É um método para extrair recursos de sentenças de texto e pode ser utilizado para algoritmos de aprendizado de máquina de treinamento. Ele cria um vocabulário de todas as palavras exclusivas que ocorrem em todos as sentenças do conjunto de treinamento (train dataset).

Por exemplo, se tivermos duas sentenças:

*   S1: He is a lazy boy. She is also lazy.
*   S2: Smith is a lazy person.

Primeiramente é criado um vocabulário usando as palavras únicas das sentenças

[‘He’ , ’She’ , ’lazy’ , ‘boy’ , ‘Smith’ , ’person’]

Como podemos ver, a lista não considera “is” , “a” , “also” nesse vocabulário porque eles não são informações relevantemente necessárias para o modelo



*   Temos , S=2, N=6
*   É criada uma matriz M de tamanho 2X6 representada como essa [tabela](https://drive.google.com/file/d/1sKw_nOxx50nrsb__dzV39vb5ckFVFMRp/view?usp=sharing)

Isso é chamado de abordagem por palavras, já que o número de ocorrências e não a sequência ou a ordem das palavras é importante nessa abordagem.






In [0]:
# Temos um pacote chamado CountVectorizer para realizar essa tarefa

from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')

# Matriz do recurso bag-of-words

bow = bow_vectorizer.fit_transform(combine['Tidy_Tweets'])

df_bow = pd.DataFrame(bow.todense())

df_bow

### **TF-IDF**

Também conhecido como Frequência do termo - frequência inversa do documento é um peso frequentemente usado na recuperação de informações e na mineração de texto. Esse peso é uma medida estatística usada para avaliar a importância de uma palavra para uma sentença em um dataset. A importância aumenta proporcionalmente ao número de vezes que uma palavra aparece na sentença, mas é compensada pela frequência da palavra no dataset. Esse peso é divido em dois termos:

**Frequência do termo (TF):**

TF = N° de vezes que o termo aparece em uma sentença / total n° de termos na sentença

**Frequência inversa do documento (IDF):**

IDF = log (n° total de sentenças / n° de sentenças com o termo)

**TF-IDF:**

TF-IDF = TF * IDF

In [0]:
# Temos um pacote do Scikit-Learn que executa esse cálculo, o TfidfVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(max_df=0.90, min_df=2,max_features=1000,stop_words='english')

tfidf_matrix=tfidf.fit_transform(combine['Tidy_Tweets'])

df_tfidf = pd.DataFrame(tfidf_matrix.todense())

df_tfidf

Estas são as técnicas de incorporação de palavras que usamos em nosso dataset para extração de recursos.



### **Separando dataset em treino e validação**

Agora temos um dataset com os recursos do modelo bag-of-words (df_bow) e outro dataset com os recursos do modelo TF-IDF (df_tfidf). A primeira tarefa é dividir o dataset em treino e validação para que possamos treinar e testar nosso modelo antes de aplicá-lo para prever dados de teste não vistos e não rotulados.

In [0]:
# Usando recursos do bag-of-words para o set de treinamento

train_bow = bow[:31962]

train_bow.todense()

In [0]:
# Usando recursos do TF-IDF para o set de treinamento 

train_tfidf_matrix = tfidf_matrix[:31962]

train_tfidf_matrix.todense()

In [0]:
# Dividindo os dados em set de treinamento e validação

from sklearn.model_selection import train_test_split

In [0]:
# Recursos do bag-of-words

x_train_bow, x_valid_bow, y_train_bow, y_valid_bow = train_test_split(train_bow,train['label'],test_size=0.3,random_state=2)

In [0]:
# Recursos do TF-IDF

x_train_tfidf, x_valid_tfidf, y_train_tfidf, y_valid_tfidf = train_test_split(train_tfidf_matrix,train['label'],test_size=0.3,random_state=17)

Terminamos com a separação dos dataset de treino e validação e agora vamos para parte mais esperada de toda atividade! \0/

### **Aplicando modelos de aprendizado de máquina**

O problema que nos propusemos a resolver está incluido na categoria de aprendizado de máquina supervisionado. A maioria dos exemplos práticos de aprendizado de máquina serão de aprendizagem supervisionada. Dando uma breve descrição:

O aprendizado supervisionado é onde você tem variáveis ​​de entrada (x) e uma variável de saída (Y) e usa um algoritmo para aprender a função de mapeamento da entrada para a saída.

Y = f(X)

O objetivo é aproximar tão bem a função de mapeamento que, quando você tiver novos dados de entrada (x), poderá prever as variáveis ​​de saída (Y) para esses dados.

Problemas de aprendizado supervisionado podem ser agrupados em problema de regressão e classificação. O nosso problema está no grupo de classificação porque temos que determinar o resultado avaliado em positivo ou negativo.

Geralmente, usamos modelos diferentes para ver qual melhor se ajusta ao nosso dataset e, em seguida, usamos esse modelo para prever resultados nos dados de teste.

Aqui vamos usar 3 modelos diferentes:

*   Logistic Regression
*   XGBoost
*   Decision Trees

Utilizaremos F1 Score durante todo processo para avaliar o desempenho do nosso modelo em vez de precisão (Accuracy).



In [0]:
# Importando f1 score

from sklearn.metrics import f1_score

**Logistic Regression**

In [0]:
# Importando o modelo de logistic regression

from sklearn.linear_model import LogisticRegression

# Ajustando o modelo de Logistic Regression

Log_Reg = LogisticRegression(random_state=0,solver='lbfgs')

Utilização com o recurso de bag-of-words:

In [0]:
# Utilizando recurso de bag-of-words

Log_Reg.fit(x_train_bow,y_train_bow)

In [0]:
# Preditando as probabilidades

prediction_bow = Log_Reg.predict_proba(x_valid_bow)

prediction_bow

In [0]:
# Calculando o f1 Score

# Se a predição é maior que ou igual a 0.3 então 1 se não 0
# Onde 0 é para tweets de sentimento positivo e 1 para negativos

prediction_int = prediction_bow[:,1]>=0.3

# Convertendo os resultados para int

prediction_int = prediction_int.astype(np.int)
prediction_int

# Calculando o f1 score
log_bow = f1_score(y_valid_bow, prediction_int)

log_bow

Utilização com o recurso de TF-IDF:

In [0]:
# Utilizando recurso de TF-IDF

Log_Reg.fit(x_train_tfidf,y_train_tfidf)

In [0]:
# Preditando as probabilidades

prediction_tfidf = Log_Reg.predict_proba(x_valid_tfidf)

prediction_tfidf

In [0]:
# Calculando o f1 Score

# Se a predição é maior que ou igual a 0.3 então 1 se não 0
# Onde 0 é para tweets de sentimento positivo e 1 para negativos

prediction_int = prediction_tfidf[:,1]>=0.3

# Convertendo os resultados para int

prediction_int = prediction_int.astype(np.int)
prediction_int

# Calculando o f1 score

log_tfidf = f1_score(y_valid_tfidf, prediction_int)

log_tfidf

**XGBoost**

In [0]:
# Importando o modelo de XGBoost

from xgboost import XGBClassifier

# Ajustando o modelo de XGBoost

model_bow = XGBClassifier(random_state=22,learning_rate=0.9)
model_tfidf = XGBClassifier(random_state=29,learning_rate=0.7)

Utilização com o recurso de bag-of-words: 

In [0]:
# Utilizando recurso bag-of-words

model_bow.fit(x_train_bow, y_train_bow)

In [0]:
# Preditando as probabilidades

xgb = model_bow.predict_proba(x_valid_bow)

xgb

In [0]:
# Calculando o f1 Score

# Se a predição é maior que ou igual a 0.3 então 1 se não 0
# Onde 0 é para tweets de sentimento positivo e 1 para negativos

xgb=xgb[:,1]>=0.3

# Convertendo os resultados para int

xgb_int=xgb.astype(np.int)

# Calculando o f1 score

xgb_bow=f1_score(y_valid_bow,xgb_int)

xgb_bow

Utilização com o recurso de TF-IDF:

In [0]:
# Utilizando recurso TF-IDF

model_tfidf.fit(x_train_tfidf, y_train_tfidf)

In [0]:
# Preditando as probabilidades

xgb_tfidf=model_tfidf.predict_proba(x_valid_tfidf)

xgb_tfidf

In [0]:
# Calculando o f1 Score

# Se a predição é maior que ou igual a 0.3 então 1 se não 0
# Onde 0 é para tweets de sentimento positivo e 1 para negativos

xgb_tfidf=xgb_tfidf[:,1]>=0.3

# Convertendo os resultados para int

xgb_int_tfidf=xgb_tfidf.astype(np.int)

# Calculando o f1 score

score=f1_score(y_valid_tfidf,xgb_int_tfidf)

score

**Decision Trees**

In [0]:
# Importando o modelo de Decision Trees

from sklearn.tree import DecisionTreeClassifier

# Ajustando o modelo de Decision Tree

dct = DecisionTreeClassifier(criterion='entropy', random_state=1)

Utilização com o recurso de bag-of-words: 

In [0]:
# Utilizando recurso bag-of-words

dct.fit(x_train_bow,y_train_bow)

In [0]:
# Preditando as probabilidades

dct_bow = dct.predict_proba(x_valid_bow)

dct_bow

In [0]:
# Calculando o f1 Score

# Se a predição é maior que ou igual a 0.3 então 1 se não 0
# Onde 0 é para tweets de sentimento positivo e 1 para negativos

dct_bow=dct_bow[:,1]>=0.3

# Convertendo os resultados para int

dct_int_bow=dct_bow.astype(np.int)

# Calculando o f1 score

dct_score_bow=f1_score(y_valid_bow,dct_int_bow)

dct_score_bow

Utilização com o recurso de TF-IDF:

In [0]:
# Utilizando recurso TF-IDF

dct.fit(x_train_tfidf,y_train_tfidf)

In [0]:
# Preditando as probabilidades

dct_tfidf = dct.predict_proba(x_valid_tfidf)

dct_tfidf

In [0]:
# Calculando o f1 Score

# Se a predição é maior que ou igual a 0.3 então 1 se não 0
# Onde 0 é para tweets de sentimento positivo e 1 para negativos

dct_tfidf=dct_tfidf[:,1]>=0.3

# Convertendo os resultados para int
dct_int_tfidf=dct_tfidf.astype(np.int)

# Calculando o f1 score
dct_score_tfidf=f1_score(y_valid_tfidf,dct_int_tfidf)

dct_score_tfidf

### **Comparando modelos**

Agora, vamos comparar os diferentes modelos que aplicamos em nosso conjunto de dados com diferentes técnicas de incorporação de palavras.

**Bag-of-Words**

In [0]:
Algo_1 = ['LogisticRegression(Bag-of-Words)','XGBoost(Bag-of-Words)','DecisionTree(Bag-of-Words)']

score_1 = [log_bow,xgb_bow,dct_score_bow]

compare_1 = pd.DataFrame({'Model':Algo_1,'F1_Score':score_1},index=[i for i in range(1,4)])

compare_1.T

Gerando gráfico de comparação

In [0]:
plt.figure(figsize=(18,5))

sns.pointplot(x='Model',y='F1_Score',data=compare_1)

plt.title('Bag-of-Words')
plt.xlabel('MODEL')
plt.ylabel('SCORE')

plt.show()

**TF-IDF**

In [0]:
Algo_2 = ['LogisticRegression(TF-IDF)','XGBoost(TF-IDF)','DecisionTree(TF-IDF)']

score_2 = [log_tfidf,score,dct_score_tfidf]

compare_2 = pd.DataFrame({'Model':Algo_2,'F1_Score':score_2},index=[i for i in range(1,4)])

compare_2.T

Gerando gráfico de comparação

In [0]:
plt.figure(figsize=(18,5))

sns.pointplot(x='Model',y='F1_Score',data=compare_2)

plt.title('TF-IDF')
plt.xlabel('MODEL')
plt.ylabel('SCORE')

plt.show()

Como podemos ver, o melhor modelo possível de Bag-of-Words e TF-IDF é Logistic Regression. Agora, vamos comparar a pontuação do modelo de Logistic Regression com as técnicas de extração de recursos que são Bag-of-Words e TF-IDF.

In [0]:
Algo_best = ['LogisticRegression(Bag-of-Words)','LogisticRegression(TF-IDF)']

score_best = [log_bow,log_tfidf]

compare_best = pd.DataFrame({'Model':Algo_best,'F1_Score':score_best},index=[i for i in range(1,3)])

compare_best.T

Gerando gráfico de comparação

In [0]:
plt.figure(figsize=(18,5))

sns.pointplot(x='Model',y='F1_Score',data=compare_best)

plt.title('Logistic Regression(Bag-of-Words & TF-IDF)')
plt.xlabel('MODEL')
plt.ylabel('SCORE')

plt.show()

Fica claro então que a melhor possibilidade para o F1 Score é utilizando o modelo Logistic Regression com o recurso de TF-IDF

In [0]:
test_tfidf = tfidf_matrix[31962:]
test_pred = Log_Reg.predict_proba(test_tfidf)

test_pred_int = test_pred[:,1] >= 0.3
test_pred_int = test_pred_int.astype(np.int)

test['label'] = test_pred_int

submission = test[['id','label']]
submission.to_csv('result.csv', index=False)

**Resultado da predição**

In [0]:
res = pd.read_csv('result.csv')res